Note: Enter the code block and hit Ctrl+Enter to execute it

In [1]:
import numpy as np
import multicomplex as mcx
from scipy.special import factorial

R = 8.314462618 # J/mol/K

# Values for argon
Tc = 150.687 # K
pc = 4863000.0 # Pa

class vdWEOS:
    a = (27/64)*(R*Tc)**2/pc
    b = (1/8)*(R*Tc)/pc

    def alphar(self, T, rho): 
        return -np.log(1.0-self.b*rho)-self.a*rho/(R*T)

# Instance of the class
vdW = vdWEOS()

# This state point
T = 300 # K
rho = 1.3 # mol/m^3

# Pressure
dalphardrho = mcx.diff_mcx1(lambda r: vdW.alphar(T, r), rho, 1, True)[1]
p = rho*R*T*(1+rho*dalphardrho)
print(f'p: {p} Pa')

# The first few virial coefficients
# ...
# Derivatives of alphar at zero density up to fourth order
ders = mcx.diff_mcx1(lambda r: vdW.alphar(T, r), 0.0, 4, True)
for n in range(2,5):
    B_n = ders[n-1]/factorial(n-2)
    units = f'm^{3*(n-1)}/mol^{(n-1)}'
    print(f'B_{n}: {B_n} {units}')

# Isochoric tension
# ...
# First derivative of alphar w.r.t. T at constant density
dalphardT = mcx.diff_mcx1(lambda T_: vdW.alphar(T_, rho), T, 1, True)[1]
# Cross (density, temperature) derivative of alphar
d2alphardTdrho = mcx.diff_mcxN(lambda x: vdW.alphar(x[0], x[1]), [T, rho], [1, 1])
betaV = (1 + rho*dalphardT + rho*T*d2alphardTdrho)*(rho*R)
print(f'betaV: {betaV} Pa/K')

p: 3242.546045484618 Pa
B_2: -2.238945068494697e-05 m^3/mol^1
B_3: 1.0371257814774649e-09 m^6/mol^2
B_4: 3.340005219645591e-14 m^9/mol^3
betaV: 10.809571850439895 Pa/K
